In [1]:
%%capture
!pip install transformers datasets

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm
from transformers import BertTokenizer

This is a template of the notebook that you should complete and enrich with your own code.

First cells will be the same than the ones of the lab on text convolution.

# Data loading


In [3]:
dataset = load_dataset("scikit-learn/imdb", split="train")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


# Pre-processing / Tokenization

This is a very important step. It maybe boring but very important. In this session we will be lazy, but in real life, the time spent on inspecting and cleaning data is never wasted. It is true for text, but also for everything.



In PyTorch, everything is tensor. Words are replaced by indices. A sentence, is therefore a sequence of indices (long integers). In the first HW, you constructed a `WhiteSpaceTokenizer`. Here we will use an already built tokenizer. It is more appropriate to transformers. It relies on sub-word units, and converts everything in lower case. This is not always the best choice, but here it will be sufficient. To quote the documentation, this tokenizer allows you to:
- Tokenize (splitting strings in sub-word token strings), converttokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).
- Add new tokens to the vocabulary in a way that is independent of the underlying structure (BPE, SentencePiece…).
- Manage special tokens (like mask, beginning-of-sentence, etc.): adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization.

Here we are going to use the tokenizer from the well known Bert model, that we can directly download.

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def preprocessing_fn(x, tokenizer):
    x["review_ids"] = tokenizer(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )["input_ids"]
    x["label"] = 0 if x["sentiment"] == "negative" else 1
    return x


Same celel than in the lab session.

🚧 **TODO** 🚧

Read the documentation about HuggingFace dataset and complete the code below.
You should:
- Shuffle the dataset
- For computational reasons, use only a total of **5000 samples**.
- Tokenize the dataset with the `preprocessing_fn`. (*Hint: use the `Dataset.map` method from HuggingFace*).
- Keep only columns `review_ids` and `label`.
- Make a train/validation split, (**80% / 20%**). Call these dataset `train_set` and `valid_set`.


## Q1

In [6]:
n_samples = 5000  # the number of training example

# We first shuffle the data !
data_shuffled = dataset.shuffle()

# Select 5000 samples
data_shuffled_sampled = data_shuffled.select(range(n_samples))

# Tokenize the dataset
data_tokenized = data_shuffled_sampled.map(lambda x: preprocessing_fn(x, tokenizer))

# Remove useless columns
data_tokenized = data_tokenized.remove_columns(["review", "sentiment"])

# Split the train and validation
train_set = data_tokenized.train_test_split(test_size=0.2)["train"]
valid_set = data_tokenized.train_test_split(test_size=0.2)["test"]

document_train_set = train_set["review_ids"]
document_valid_set = valid_set["review_ids"]


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [8]:
data_tokenized[0]

{'review_ids': [2023,
  2522,
  6508,
  2690,
  2465,
  13130,
  2150,
  1996,
  3395,
  1997,
  2172,
  11150,
  2011,
  1996,
  2047,
  8843,
  1997,
  10904,
  25119,
  1999,
  1996,
  3865,
  1010,
  2107,
  2004,
  3841,
  19127,
  1012,
  2106,
  2009,
  10107,
  2107,
  28238,
  1029,
  2092,
  6609,
  1998,
  2238,
  2001,
  2196,
  12266,
  1010,
  2009,
  2001,
  2196,
  1037,
  2092,
  1011,
  2517,
  6925,
  2007,
  2367,
  16457,
  2029,
  29261,
  2012,
  1996,
  7091,
  1012,
  2009,
  2001,
  1996,
  2265,
  2115,
  2388,
  1998,
  2269,
  1998,
  2763,
  14472,
  3427,
  1012,
  2027,
  10252,
  2738,
  2084,
  4191,
  1010,
  2021,
  2027,
  2020,
  2196,
  3497,
  2000,
  2022,
  15807,
  2011,
  1996,
  2265,
  1010,
  2061,
  1996,
  2755,
  2008,
  2009,
  2196,
  8315,
  2068,
  2001,
  10047,
  8585,
  14482,
  1012,
  1026,
  7987,
  1013,
  1028,
  1026,
  7987,
  1013,
  1028,
  2028,
  3329,
  1999,
  1996,
  6542,
  4265,
  3322,
  2138,
  2009,
  2596,
  2

# Q2

In [25]:
#A revoir car les C n'ont pas toujours la même taille, pour l'instant je l'utilise pour avancer

def extract_words_contexts(w, R):
  ids = []
  pos_con = []
  for i in range(len(w)):
    ids.append(w[i])
    if i<R:
      pos_con.append(w[0:i])
    else:
      pos_con.append(w[i-R:i])
    if i>len(w)-R-1:
      pos_con.append(w[i+1:len(w)])
    else:
      pos_con.append(w[i+1:i+R+1])
  return ids, pos_con

# Q3

In [26]:
def flatten_dataset_to_list(dataset, R):
  L=[]
  for x in dataset:
    L.append(extract_words_contexts(x, R))
  return L

In [11]:
data_tokenized_flattened = flatten_dataset_to_list(data_tokenized["review_ids"], 10)

In [12]:
data_tokenized_flattened[0]

([2023,
  2522,
  6508,
  2690,
  2465,
  13130,
  2150,
  1996,
  3395,
  1997,
  2172,
  11150,
  2011,
  1996,
  2047,
  8843,
  1997,
  10904,
  25119,
  1999,
  1996,
  3865,
  1010,
  2107,
  2004,
  3841,
  19127,
  1012,
  2106,
  2009,
  10107,
  2107,
  28238,
  1029,
  2092,
  6609,
  1998,
  2238,
  2001,
  2196,
  12266,
  1010,
  2009,
  2001,
  2196,
  1037,
  2092,
  1011,
  2517,
  6925,
  2007,
  2367,
  16457,
  2029,
  29261,
  2012,
  1996,
  7091,
  1012,
  2009,
  2001,
  1996,
  2265,
  2115,
  2388,
  1998,
  2269,
  1998,
  2763,
  14472,
  3427,
  1012,
  2027,
  10252,
  2738,
  2084,
  4191,
  1010,
  2021,
  2027,
  2020,
  2196,
  3497,
  2000,
  2022,
  15807,
  2011,
  1996,
  2265,
  1010,
  2061,
  1996,
  2755,
  2008,
  2009,
  2196,
  8315,
  2068,
  2001,
  10047,
  8585,
  14482,
  1012,
  1026,
  7987,
  1013,
  1028,
  1026,
  7987,
  1013,
  1028,
  2028,
  3329,
  1999,
  1996,
  6542,
  4265,
  3322,
  2138,
  2009,
  2596,
  2012,
  2034,
 

In [13]:
R, K = 10, 2

# Q4

In [14]:
document_train_set = flatten_dataset_to_list(train_set["review_ids"], R)
document_valid_set = flatten_dataset_to_list(valid_set["review_ids"], R)

In [15]:
print(len(document_train_set))
print(len(document_valid_set))

4000
1000


# Q5

In [27]:
from torch.utils.data import Dataset


class sentiment(Dataset):
    def __init__(self, data):
      self.data =data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        return self.data[idx]

In [28]:
train_set = sentiment(document_train_set)
valid_set = sentiment(document_valid_set)

# Q6

In [34]:
#J'y arrive pas ^^'
import random
import itertools
def collate_fn(batch, R, K):
  ids, poss_con, negs_con = [], [], []
  batch = extract_words_contexts(batch, R)
  for word_id, pos_con in batch:
    ids.append(word_id)
    poss_con.append(pos_con)
    neg_con = random.sample(data_tokenized_flattened, 2*K*R)
    negs_con.append(neg_con)

  ids = list(itertools.chain(*ids))
  word_id = torch.tensor(ids)

  poss_con = list(itertools.chain(*poss_con))
  positive_context_ids = torch.tensor(poss_con)

  negative_context_ids = list(itertools.chain(*negs_con))
  #negative_context_ids = torch.tensor(negs_con)

  return {"word_id":ids, "positive_context_ids":positive_context_ids, "negative_context_ids": negative_context_ids}


In [35]:
collate_fn(valid_set, R, K)["positive_context_ids"]

ValueError: expected sequence of length 256 at dim 1 (got 512)

# Q7

In [32]:
batch_size = 10
dataloader = DataLoader(
        dataset=data_tokenized_flattened, batch_size=batch_size, collate_fn=lambda x: collate_fn(x, R, K)
    )

In [33]:
for i, batch in enumerate(dataloader):
    print(f"R = {R}, K = {K}")
    #print(f"Word IDs shape: {batch['word_id'].shape}")
    #print(f"Positive Context IDs shape: {batch['positive_context_ids'].shape}")
    #print(f"Negative Context IDs shape: {batch['negative_context_ids'].shape}")
    print("Word IDs:", batch['word_id'])
    print("Positive Context:", batch['positive_context_ids'])
    print("Negative Context:", batch['negative_context_ids'])
    if i == 2:
      break


ValueError: expected sequence of length 217 at dim 1 (got 434)